# Demo with several models and words

In [4]:
from huggingface_hub import login
from openai import OpenAI
import os
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
import matplotlib.pyplot as plt

c:\Users\zulfi\Desktop\Human AI Interaction\Graded Project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Set-ups

In [5]:
os.environ["NEBIUS_API_KEY"] = "v1.CmMKHHN0YXRpY2tleS1lMDByNG13OHJlOTEwYXhtZjcSIXNlcnZpY2VhY2NvdW50LWUwMHZ2ZW53eDUwMDM1NTU1NjIMCOeCosgGEPu9g74COgsI54W6kwcQwMLuSkACWgNlMDA.AAAAAAAAAAF-s3IVuPd-6SwZfzos0vgqlAlUZtfge6Kj5JAVVepABWajqetR76LusvMMN1mo0E5Y5TbLdhzBkjxNaiMXxrQM"

In [ ]:
MODEL_CONFIG = {
    "weak":   "meta-llama/Meta-Llama-3.1-8B-Instruct",
    "avg":    "google/gemma-2-9b-it-fast",         
    "strong": "openai/gpt-oss-20b"
}

In [27]:
def build_model(model_name):
    return ChatOpenAI(
        base_url="https://api.studio.nebius.ai/v1",
        api_key=os.getenv("NEBIUS_API_KEY"),
        model=model_name,
        temperature=0.7
    )

models = { role: build_model(name) for role, name in MODEL_CONFIG.items() }

model_order = ["weak", "avg", "strong"]

## Utility functions

In [6]:
from sentence_transformers import SentenceTransformer, util

embedder = SentenceTransformer("all-MiniLM-L6-v2")

def similarity_curve(original_word, chain):
    orig_emb = embedder.encode(original_word, convert_to_tensor=True)
    sims = []
    for w in chain:
        emb = embedder.encode(w, convert_to_tensor=True)
        sims.append(util.cos_sim(orig_emb, emb).item())
    return sims

In [7]:
def evaluation (common_chain, common_word, rare_chain, rare_word):
    # rare_word = "dates"
    common_sims = similarity_curve(common_word, common_chain)
    rare_sims = similarity_curve(rare_word, rare_chain)
    plt.figure(figsize=(8,5))
    plt.plot(common_sims, label=f"Common: {common_word}")
    plt.plot(rare_sims, label=f"Rare: {rare_word}")
    plt.xlabel("Chain Step")
    plt.ylabel("Semantic Similarity to Original")
    plt.title("Semantic Drift: Rare vs. Common Word")
    plt.legend()
    plt.grid(True)
    plt.show()

## Chain with some memory with single model involved

In [14]:
def run_chain_with_minimal_memory(model, initial_word, steps=2):
    word = initial_word
    chain = [word]
    chain_desc = []

    # Step 0: first description
    description = model.invoke([
        HumanMessage(f"Describe this word in 2 sentences without naming it: {word}.")
    ]).content
    chain_desc.append(description)

    for _ in range(steps):
        # 1) Person guesses the word from the description
        guess = model.invoke([
            HumanMessage(
                f"You recieved this description: \"{description}\".\n"
                f"Guess the word. Reply with ONLY the guessed word."
            )
        ]).content.strip()

        chain.append(guess)

        # 2) Person creates a NEW description based on:
        #    - the description they heard
        #    - their guessed word
        new_description = model.invoke([
            HumanMessage(
                f"You just heard this description: \"{description}\".\n"
                f"You guessed the word: \"{guess}\".\n"
                f"Create another description for your guessed word in 2 sentences "
                f"WITHOUT naming it directly."
            )
        ]).content

        chain_desc.append(new_description)

        # Move forward
        word = guess
        description = new_description

    return chain, chain_desc


In [15]:
initial_word = "kiwi"
steps = 3

chain, chain_desc = run_chain_with_minimal_memory(
    model=models["llama3_8b"],   # <-- 2nd model from your dictionary
    initial_word=initial_word,
    steps=steps
)

print("\n=== Chain Words ===")
for i, w in enumerate(chain):
    print(f"{i}: {w}")

print("\n=== Descriptions ===")
for i, d in enumerate(chain_desc):
    print(f"{i}: {d}")



=== Chain Words ===
0: kiwi
1: Kiwi
2: Kiwi
3: Dragon Fruit

=== Descriptions ===
0: This small, fuzzy fruit has a vibrant green skin and a sweet, slightly tart flavor when eaten ripe. Its soft, pulpy interior is often fuzzy and has tiny black seeds, and it's a popular ingredient in smoothies, salads, and desserts.
1: This small, tart fruit has a brown, fuzzy skin that's hard to peel, but its vibrant green flesh is a burst of flavor when eaten ripe. Its tiny black seeds and stringy, fuzzy texture make it a fun ingredient to add to oatmeal, yogurt parfaits, and other breakfast treats.
2: This exotic, tropical fruit has a fuzzy, brown exterior that's often difficult to remove, but its sweet and tangy green interior is a refreshing surprise. When ripe, its tiny black seeds and stringy pulp add a fun texture to smoothies, salads, and desserts, making it a popular ingredient in many cuisines.
3: This vibrant, pink or yellow fruit has a spiky, green exterior that's often a conversation star

## Chain with some memory several models involved

In [ ]:
def one_chain_block(model, chain, chain_desc):
    description = chain_desc[-1]
    guess = model.invoke([
        HumanMessage(
            f"You recieved this description: \"{description}\".\n"
            f"Guess the word. Reply with ONLY the guessed word."
        )
    ]).content.strip()

    chain.append(guess)

    # Person creates a NEW description based on:
    # 1) the description they heard
    # 2) their guessed word

    new_description = model.invoke([
        HumanMessage(
            f"You just heard this description: \"{description}\".\n"
            f"You guessed the word: \"{guess}\".\n"
            f"Create another description for your guessed word in ONLY 2 sentences."
            f"WITHOUT naming it directly."
        )
    ]).content

    chain_desc.append(new_description)

    return chain, chain_desc

In [ ]:
word = "papaya"
chain = [word]
chain_desc = []

description = models["avg"].invoke([
    HumanMessage(f"Describe this word in 3 sentences without naming it: {word}.")
]).content
chain_desc.append(description)

for role in model_order:
    model = models[role]
    chain, chain_desc = one_chain_block(model, chain, chain_desc)


In [30]:
for i, (w, d, role) in enumerate(zip(chain, chain_desc, model_order)):
    print(f"--- Step {i} ({role}) ---")
    print(f"Word: {w}")
    print(f"Description:\n{d}\n")

--- Step 0 (weak) ---
Word: papaya
Description:
This tropical fruit boasts a vibrant, orange flesh and a sweet, slightly musky flavor.  Its large, oval shape often conceals a central core filled with black seeds. 




--- Step 1 (avg) ---
Word: Pineapple
Description:
This prickly, tropical superstar boasts a tough, waxy exterior that gives way to a juicy, tangy interior. Its tough, fibrous leaves are a distinctive feature, and its sweet, tropical flavor is a staple of summer desserts and salads.

--- Step 2 (strong) ---
Word: Pineapple
Description:
This fruit, a symbol of hospitality,  is crowned with a leafy rosette and hidden within a tough shell.  Its golden flesh, when sliced, reveals a unique, complex sweetness that can be enjoyed both raw and cooked.  





